In [1]:
import watertap as wt
import pandas as pd
from pyomo.environ import value, Block
from idaes.core import FlowsheetBlock
import numpy as np
from case_study_trains import *
import case_study_trains
import pyomo.environ as env
from pyomo.environ import Constraint
m = wt.watertap_setup(dynamic = False)


wt.case_study_trains.train = {"case_study": "uranium",
                             "reference": "nawi",
                             "scenario": "baseline"}


wt.case_study_trains.source_water = {"case_study": "uranium", 
                             "reference": "nawi",
                             "scenario": "baseline",
                             "water_type": "uranium_ro_influent"}

m = wt.case_study_trains.get_case_study(m=m) # flow is set as case study flow unless defined.



uranium
------- Adding Unit Processes -------
well_field
ro_first_pass
ro_second_pass
treated_storage
21600.0
mining
deep_well_injection
-------------------------------------
adding source
----- Connecting Unit Processes -----
uranium_ro_influent ToUnitName --> well_field
well_field ToUnitName --> ro_first_pass
ro_first_pass ToUnitName --> ro_second_pass
ro_second_pass ToUnitName --> deep_well_injection
treated_storage ToUnitName --> mining
ro_first_pass ToUnitName --> mixer1
ro_second_pass ToUnitName --> mixer1
mixer1 ToUnitName --> treated_storage
-------------------------------------


In [2]:
wt.display.show_train2(model_name=m)

In [3]:
# # RUN MODEL with optimal ro --> estimating area and pressure for optimal LCOW
# so that the model solves and gets you results. Then runs again with set pressure.
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).feed.pressure.unfix()
        getattr(m.fs, key).membrane_area.unfix()
        print("unfixing feed presure and area for", key)
        
wt.run_water_tap(m = m, objective=True, skip_small=False)

# prints RO results
print("optimal ro area and pressures:")
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

# RESET PRESSURE TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        if "feed_pressure" in m.fs.pfd_dict[key]["Parameter"]:
            if m.fs.pfd_dict[key]["Parameter"]["type"] == "pass":
                getattr(m.fs, key).feed.pressure.fix(m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])
                print("setting feed presure for", key, "to -->", m.fs.pfd_dict[key]["Parameter"]["feed_pressure"])

wt.run_water_tap(m = m, objective=True, print_model_results="summary", skip_small = True)

for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

initial inflow to train is relatively small (< 1 m3/s). running model with dummy flows to initialize.
degrees_of_freedom: 0
WaterTAP3 solution optimal
model finished running to initialize conditions. now running with actual inflows.
degrees_of_freedom: 0
WaterTAP3 solution optimal
optimal ro area and pressures:
degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.well_field
total_cap_investment: 0.16628966659706035
----------------------------------------------------------------------
fs.ro_first_pass
total_cap_investment: 3.2603360838333018
----------------------------------------------------------------------
fs.ro_second_pass
total_cap_investment: 0.026405437163256346
----------------------------------------------------------------------
fs.treated_storage
total_cap_investment: 0.20996945948582046
----------------------------------------------------------------------
fs.mining
total_cap_investment: 0.0289332028277115
-------------------------------------------------------------------

In [4]:
# If you need the system recovery to match better.... set a maximum recovery rate.

#m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.55) # THIS IS FOR TAMPA BAY
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery <= 0.50) # THIS IS FOR SANTA BARBARA

In [5]:
# set cap utilization factor
m.fs.costing_param.plant_cap_utilization = 1

In [6]:
wt.run_water_tap(m = m, objective=True, print_model_results="summary", skip_small = True)
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        print(key, "feed pressure", getattr(m.fs, key).feed.pressure[0]())
        print(key, "membrane area", getattr(m.fs, key).membrane_area[0]())

degrees_of_freedom: 0
WaterTAP3 solver returned an infeasible solution
Running again with updated initial conditions --- attempt 1
WaterTAP3 solver returned an infeasible solution
Running again with updated initial conditions --- attempt 2
WaterTAP3 solver returned an infeasible solution
Running again with updated initial conditions --- attempt 3
WaterTAP3 solution infeasible
WaterTAP3 solver returned an infeasible FINAL solution. Check option to run model with updated initial conditions
fs.well_field
total_cap_investment: 0.16628966551825666
----------------------------------------------------------------------
fs.ro_first_pass
total_cap_investment: 3.0689617261930415
----------------------------------------------------------------------
fs.ro_second_pass
total_cap_investment: 0.025837703192610417
----------------------------------------------------------------------
fs.treated_storage
total_cap_investment: 0.19769331407456625
----------------------------------------------------------

In [7]:
# RESET AREA TO USER INPUT
for key in m.fs.pfd_dict.keys():
    if m.fs.pfd_dict[key]["Unit"] == "reverse_osmosis":
        getattr(m.fs, key).membrane_area.fix(getattr(m.fs, key).membrane_area[0]())

In [8]:
# Readjust recovery constraint and deactivate objective constraint
m.recovery_bound = Constraint(expr = m.fs.costing.system_recovery >= 0)
m.fs.objective_function.deactivate()

In [9]:
# NOW RUN AS SIMULATION
wt.run_water_tap(m = m, objective=False, print_model_results="summary", skip_small = True)

degrees_of_freedom: 0
WaterTAP3 solution optimal
fs.well_field
total_cap_investment: 0.16628966659706035
----------------------------------------------------------------------
fs.ro_first_pass
total_cap_investment: 3.2603360838333018
----------------------------------------------------------------------
fs.ro_second_pass
total_cap_investment: 0.026405437194112563
----------------------------------------------------------------------
fs.treated_storage
total_cap_investment: 0.20996945948753612
----------------------------------------------------------------------
fs.mining
total_cap_investment: 0.0289332028279954
----------------------------------------------------------------------
fs.deep_well_injection
total_cap_investment: 9.03898786212338
----------------------------------------------------------------------
----------------------------------------------------------------------
------------------- System Level Metrics and Costs -------------------
Total Capital Investment ($MM) 12.

In [10]:
# creates csv in results folder with the name: *case_study*_*scenario*.csv
# In this case, save the final baseline result.

df = wt.get_results_table(m = m, case_study = wt.case_study_trains.source_water["case_study"], 
                                scenario = wt.case_study_trains.source_water["scenario"])